In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [3]:
train=pd.read_csv(r"C:\Users\yjhut\OneDrive\바탕 화면\데이콘 감귤 rmse\train.csv")
test=pd.read_csv(r"C:\Users\yjhut\OneDrive\바탕 화면\데이콘 감귤 rmse\test.csv")

In [4]:
sub_m=pd.read_csv(r"C:\Users\yjhut\OneDrive\바탕 화면\데이콘 감귤 rmse\sample_submission.csv")

In [5]:
def resumtable(df):
    print(f'데이터 형상',{df.shape})
    summary=pd.DataFrame(df.dtypes, columns=['데아터 타입'])
    summary=summary.reset_index()
    summary=summary.rename(columns={'index':'feature명'})
    summary['고유값 개수']= df.nunique().values
    summary['결측값 개수']= df.isnull().sum().values
    return summary

In [62]:
resumtable(train)

데이터 형상 {(2207, 183)}


,feature명,데아터 타입,고유값 개수,결측값 개수
0,착과량(int),int64,737,0
1,수고(m),float64,205,0
2,수관폭1(min),float64,320,0
3,수관폭2(max),float64,336,0
4,수관폭평균,float64,558,0
...,...,...,...,...
178,2022-11-24 엽록소,float64,2207,0
179,2022-11-25 엽록소,float64,2207,0
180,2022-11-26 엽록소,float64,2207,0
181,2022-11-27 엽록소,float64,2207,0


In [66]:
for col in train.columns:
    print(f"{col}의 중앙값- 평균값: ",train[col].median()-train[col].mean())

착과량(int)의 중앙값- 평균값:  -2.2224739465337393
수고(m)의 중앙값- 평균값:  -0.7299501585863197
수관폭1(min)의 중앙값- 평균값:  0.8137743543271654
수관폭2(max)의 중앙값- 평균값:  5.175804259175379
수관폭평균의 중앙값- 평균값:  3.4947893067512723
2022-09-01 새순의 중앙값- 평균값:  0.0251925690983259
2022-09-02 새순의 중앙값- 평균값:  -0.03692795650203795
2022-09-03 새순의 중앙값- 평균값:  -0.001495242410512354
2022-09-04 새순의 중앙값- 평균값:  0.03656547349343331
2022-09-05 새순의 중앙값- 평균값:  -0.02750339827820092
2022-09-06 새순의 중앙값- 평균값:  0.009877661984597275
2022-09-07 새순의 중앙값- 평균값:  0.04671499773448273
2022-09-08 새순의 중앙값- 평균값:  -0.016130493883100616
2022-09-09 새순의 중앙값- 평균값:  0.02084277299501114
2022-09-10 새순의 중앙값- 평균값:  -0.043135478024467044
2022-09-11 새순의 중앙값- 평균값:  -0.006207521522432646
2022-09-12 새순의 중앙값- 평균값:  0.030811055731759573
2022-09-13 새순의 중앙값- 평균값:  -0.03212505663797627
2022-09-14 새순의 중앙값- 평균값:  0.004349796103307657
2022-09-15 새순의 중앙값- 평균값:  0.04073402809243465
2022-09-16 새순의 중앙값- 평균값:  -0.022745808790204602
2022-09-17 새순의 중앙값- 평균값:  0.014907113729047072
2022-

In [6]:
train=train.set_index('ID')
test=test.set_index('ID')

In [7]:
for col in train.columns:
    print(f"{col}의 중앙값- 평균값: ",train[col].skew())

착과량(int)의 중앙값- 평균값:  -0.019181543802712243
수고(m)의 중앙값- 평균값:  -0.20773824991969436
수관폭1(min)의 중앙값- 평균값:  -0.023607285844454384
수관폭2(max)의 중앙값- 평균값:  -0.10781812697384853
수관폭평균의 중앙값- 평균값:  -0.1033046699952813
2022-09-01 새순의 중앙값- 평균값:  -0.013524715658337662
2022-09-02 새순의 중앙값- 평균값:  -0.00996449212572391
2022-09-03 새순의 중앙값- 평균값:  -0.012828247303569093
2022-09-04 새순의 중앙값- 평균값:  -0.01101516689796272
2022-09-05 새순의 중앙값- 평균값:  -0.010521474909316498
2022-09-06 새순의 중앙값- 평균값:  -0.01365358056932201
2022-09-07 새순의 중앙값- 평균값:  -0.010650738352318946
2022-09-08 새순의 중앙값- 평균값:  -0.014499474239018338
2022-09-09 새순의 중앙값- 평균값:  -0.012637687583385486
2022-09-10 새순의 중앙값- 평균값:  -0.01326994341749257
2022-09-11 새순의 중앙값- 평균값:  -0.013044892803746374
2022-09-12 새순의 중앙값- 평균값:  -0.011155503674997918
2022-09-13 새순의 중앙값- 평균값:  -0.012354350546707544
2022-09-14 새순의 중앙값- 평균값:  -0.01743656571323245
2022-09-15 새순의 중앙값- 평균값:  -0.014131046237922175
2022-09-16 새순의 중앙값- 평균값:  -0.011933011442429156
2022-09-17 새순의 중앙값- 평균값:  -0.0

2022-11-19 엽록소의 중앙값- 평균값:  -0.024956222360393573
2022-11-20 엽록소의 중앙값- 평균값:  -0.024951457399739294
2022-11-21 엽록소의 중앙값- 평균값:  -0.026184261322101884
2022-11-22 엽록소의 중앙값- 평균값:  -0.02570731158065792
2022-11-23 엽록소의 중앙값- 평균값:  -0.02568554783432987
2022-11-24 엽록소의 중앙값- 평균값:  -0.025073665192789488
2022-11-25 엽록소의 중앙값- 평균값:  -0.022548939239186754
2022-11-26 엽록소의 중앙값- 평균값:  -0.021453972387225153
2022-11-27 엽록소의 중앙값- 평균값:  -0.023814239741690772
2022-11-28 엽록소의 중앙값- 평균값:  -0.025046566927293373


In [8]:
for col in test.columns:
    print(f"{col}의 중앙값- 평균값: ",test[col].skew())

수고(m)의 중앙값- 평균값:  -0.0535728823713279
수관폭1(min)의 중앙값- 평균값:  0.029275315042360382
수관폭2(max)의 중앙값- 평균값:  -0.01701182187976305
수관폭평균의 중앙값- 평균값:  -0.04249546391097111
2022-09-01 새순의 중앙값- 평균값:  -0.025852786603073098
2022-09-02 새순의 중앙값- 평균값:  -0.023565950769254725
2022-09-03 새순의 중앙값- 평균값:  -0.024588956019723916
2022-09-04 새순의 중앙값- 평균값:  -0.026221920564026925
2022-09-05 새순의 중앙값- 평균값:  -0.026368943795788087
2022-09-06 새순의 중앙값- 평균값:  -0.021963141303423295
2022-09-07 새순의 중앙값- 평균값:  -0.02051960726537159
2022-09-08 새순의 중앙값- 평균값:  -0.026756025825899133
2022-09-09 새순의 중앙값- 평균값:  -0.022577117402019942
2022-09-10 새순의 중앙값- 평균값:  -0.02659435611733883
2022-09-11 새순의 중앙값- 평균값:  -0.023837729002371424
2022-09-12 새순의 중앙값- 평균값:  -0.030256152255327615
2022-09-13 새순의 중앙값- 평균값:  -0.027863151438343132
2022-09-14 새순의 중앙값- 평균값:  -0.025970763065618227
2022-09-15 새순의 중앙값- 평균값:  -0.026959073351572254
2022-09-16 새순의 중앙값- 평균값:  -0.02644276161190918
2022-09-17 새순의 중앙값- 평균값:  -0.0289271915488918
2022-09-18 새순의 중앙값- 평균값:  

In [9]:
scale_train=train.iloc[:,1:5]

In [10]:
scale_train  # 시각화, 스케일이 필요한 데이터

,수고(m),수관폭1(min),수관폭2(max),수관폭평균
ID,,,,
TRAIN_0000,275.0,287.0,292.0,289.5
TRAIN_0001,293.0,284.0,336.0,310.0
TRAIN_0002,300.0,392.0,450.0,421.0
TRAIN_0003,289.0,368.0,379.0,373.5
TRAIN_0004,306.0,353.0,358.0,355.5
...,...,...,...,...
TRAIN_2202,230.0,353.0,367.0,360.0
TRAIN_2203,266.0,337.0,400.0,368.5
TRAIN_2204,237.0,353.0,362.0,357.5


In [11]:
n_scale_train=train.iloc[:,5:]

In [87]:
n_scale_train

,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,2022-09-05 새순,2022-09-06 새순,2022-09-07 새순,2022-09-08 새순,2022-09-09 새순,2022-09-10 새순,...,2022-11-19 엽록소,2022-11-20 엽록소,2022-11-21 엽록소,2022-11-22 엽록소,2022-11-23 엽록소,2022-11-24 엽록소,2022-11-25 엽록소,2022-11-26 엽록소,2022-11-27 엽록소,2022-11-28 엽록소
ID,,,,,,,,,,,,,,,,,,,,,
TRAIN_0000,2.8,2.8,2.7,2.7,2.7,2.7,2.6,2.6,2.6,2.6,...,70.978249,70.876794,70.705253,70.559603,70.427356,70.340491,70.293830,70.262422,70.169841,70.043251
TRAIN_0001,3.3,3.3,3.3,3.2,3.2,3.1,3.1,3.1,3.0,3.0,...,71.535483,71.382303,71.253604,71.092665,70.955608,70.796630,70.597550,70.565088,70.560502,70.427600
TRAIN_0002,3.0,2.9,2.9,2.9,2.9,2.8,2.8,2.8,2.8,2.7,...,71.279804,71.199570,71.144020,71.026740,70.920038,70.876723,70.710129,70.595971,70.418203,70.399578
TRAIN_0003,3.1,3.0,3.0,3.0,3.0,2.9,2.9,2.9,2.8,2.8,...,69.934615,69.884124,69.845683,69.794682,69.779813,69.614644,69.455404,69.281590,69.238689,69.133970
TRAIN_0004,3.7,3.6,3.6,3.6,3.5,3.5,3.5,3.4,3.4,3.3,...,68.313016,68.285364,68.209860,68.209458,68.040083,67.859963,67.775556,67.701456,67.504244,67.410093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_2202,4.7,4.6,4.5,4.5,4.4,4.4,4.3,4.3,4.2,4.2,...,66.347384,66.293609,66.193131,66.176590,66.109845,66.008775,65.889675,65.734066,65.586656,65.560547
TRAIN_2203,3.1,3.0,3.0,3.0,2.9,2.9,2.9,2.9,2.8,2.8,...,73.438844,73.432194,73.355195,73.329491,73.271361,73.088750,73.071705,73.069036,72.952559,72.944396
TRAIN_2204,3.0,3.0,2.9,2.9,2.9,2.9,2.8,2.8,2.8,2.8,...,67.576066,67.404347,67.394009,67.247428,67.117025,66.933771,66.734617,66.572095,66.506006,66.479459


In [17]:
np.round(n_scale_train.iloc[:,:89].sum(axis=1)/89,1)

ID
TRAIN_0000    1.7
TRAIN_0001    1.9
TRAIN_0002    1.8
TRAIN_0003    1.9
TRAIN_0004    2.1
             ... 
TRAIN_2202    2.5
TRAIN_2203    1.8
TRAIN_2204    1.8
TRAIN_2205    2.2
TRAIN_2206    2.8
Length: 2207, dtype: float64

In [18]:
scale_train['새순의 평균값']=np.round(n_scale_train.iloc[:,:89].sum(axis=1)/89,1)
scale_train['엽록소의 평균값']=np.round(n_scale_train.iloc[:,89:].sum(axis=1)/89,1)

In [19]:
scale_train['착과량']=train['착과량(int)']

In [20]:
scale_train

,수고(m),수관폭1(min),수관폭2(max),수관폭평균,새순의 평균값,엽록소의 평균값,착과량
ID,,,,,,,
TRAIN_0000,275.0,287.0,292.0,289.5,1.7,74.2,692
TRAIN_0001,293.0,284.0,336.0,310.0,1.9,75.0,534
TRAIN_0002,300.0,392.0,450.0,421.0,1.8,74.7,634
TRAIN_0003,289.0,368.0,379.0,373.5,1.9,73.5,639
TRAIN_0004,306.0,353.0,358.0,355.5,2.1,71.8,496
...,...,...,...,...,...,...,...
TRAIN_2202,230.0,353.0,367.0,360.0,2.5,69.8,119
TRAIN_2203,266.0,337.0,400.0,368.5,1.8,77.3,611
TRAIN_2204,237.0,353.0,362.0,357.5,1.8,71.2,650


In [21]:
scale_test=test.iloc[:,:4]

In [22]:
scale_test

,수고(m),수관폭1(min),수관폭2(max),수관폭평균
ID,,,,
TEST_0000,231.0,219.0,302.0,260.5
TEST_0001,206.0,324.0,346.0,335.0
TEST_0002,282.0,373.0,404.0,388.5
TEST_0003,234.0,337.0,357.0,347.0
TEST_0004,222.0,329.0,347.0,338.0
...,...,...,...,...
TEST_2203,217.0,265.0,288.0,276.5
TEST_2204,182.0,191.0,262.0,226.5
TEST_2205,254.0,280.0,332.0,306.0


In [23]:
no_scale_test_1=test.iloc[:,4:93]

In [24]:
no_scale_test_2=test.iloc[:,93:]

In [25]:
scale_test['새순의 평균']=np.round(no_scale_test_1.sum(axis=1)/89,1)
scale_test['엽록소의 평균']=np.round(no_scale_test_2.sum(axis=1)/89,1)

In [26]:
scale_test

,수고(m),수관폭1(min),수관폭2(max),수관폭평균,새순의 평균,엽록소의 평균
ID,,,,,,
TEST_0000,231.0,219.0,302.0,260.5,2.3,75.9
TEST_0001,206.0,324.0,346.0,335.0,1.6,71.5
TEST_0002,282.0,373.0,404.0,388.5,2.6,77.8
TEST_0003,234.0,337.0,357.0,347.0,2.1,76.9
TEST_0004,222.0,329.0,347.0,338.0,1.7,75.8
...,...,...,...,...,...,...
TEST_2203,217.0,265.0,288.0,276.5,1.5,74.2
TEST_2204,182.0,191.0,262.0,226.5,2.3,70.2
TEST_2205,254.0,280.0,332.0,306.0,2.2,66.7


In [27]:
scale_train

,수고(m),수관폭1(min),수관폭2(max),수관폭평균,새순의 평균값,엽록소의 평균값,착과량
ID,,,,,,,
TRAIN_0000,275.0,287.0,292.0,289.5,1.7,74.2,692
TRAIN_0001,293.0,284.0,336.0,310.0,1.9,75.0,534
TRAIN_0002,300.0,392.0,450.0,421.0,1.8,74.7,634
TRAIN_0003,289.0,368.0,379.0,373.5,1.9,73.5,639
TRAIN_0004,306.0,353.0,358.0,355.5,2.1,71.8,496
...,...,...,...,...,...,...,...
TRAIN_2202,230.0,353.0,367.0,360.0,2.5,69.8,119
TRAIN_2203,266.0,337.0,400.0,368.5,1.8,77.3,611
TRAIN_2204,237.0,353.0,362.0,357.5,1.8,71.2,650


In [28]:
y=scale_train['착과량']

scale_train=scale_train.drop('착과량',axis=1)

In [33]:
# 데이터 분할 
from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val=train_test_split(scale_train, y , random_state=43, test_size=0.15)

In [34]:
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((1875, 6), (332, 6), (1875,), (332,))

In [ ]:
# ==================훈련====================

In [38]:
lgb_model=lgb.LGBMRegressor(n_estimators=700, max_depth=5, random_state=43)

In [39]:
lgb_model.fit(X_tr,y_tr)

LGBMRegressor(max_depth=5, n_estimators=700, random_state=43)

In [40]:
lgb_pred=lgb_model.predict(X_val)

In [43]:
NMAE(y_val,lgb_pred)

0.11573799732983492

In [154]:
from sklearn.linear_model import LinearRegression
l_model=LinearRegression()

l_model.fit(X_tr,y_tr)

pred=l_model.predict(X_val)


#  score = 0.1009

In [41]:
##  평가지표 = NMAE (정규화 된 MAE)

def NMAE(true,pred):
    mae=np.mean(np.abs(true-pred))
    score=mae/np.mean(np.abs(true))
     
    return score

In [160]:
NMAE(pred,y_val)

0.10094083681786856

In [165]:
l_pred=l_model.predict(scale_test)

In [170]:
sub_m['착과량(int)']=l_pred

In [173]:
sub_m.to_csv('sub_mission(1226_1).csv',index=False) # 최종 예측값